In [17]:
import pandas as pd

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow as tf

# Calculate Genre encodings
(equivalent to one hot encoding?)

In [148]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/drama-notebooks/dramawiki_2020_dramas_extracted_fields.csv", sep='\t')

In [149]:
df = df[['main_name', 'genre']].dropna()

In [150]:
df.head(5)  # why is the first genre always capitalized?

,main_name,genre
0,365: Repeat the Year,"Thriller, fantasy, mystery, crime, time-travel"
1,A Piece of Your Mind,"Romance, comedy"
2,Alice (SBS),"Fantasy, science fiction, time-travel"
3,Amanza,Drama
4,Backstreet Rookie,"Romance, comedy"


In [151]:
df.shape

(111, 2)

In [153]:
training_data = df['genre'].values.tolist()
training_data[:10]

[' Thriller, fantasy, mystery, crime, time-travel',
 ' Romance, comedy',
 ' Fantasy, science fiction, time-travel',
 ' Drama',
 ' Romance, comedy',
 ' Melodrama',
 ' Drama, youth, music, romance',
 ' Drama, school',
 ' Drama, mystery',
 ' Melodrama, mystery, fantasy']

In [72]:
len(training_data)

111

In [74]:
def split_by_comma(input_string):
  return input_string.split(',')

n_genres_per_row = [len(split_by_comma(i)) for i in training_data]

In [75]:
len(n_genres_per_row)

111

In [164]:
def split_by_comma_tf(input_tensor):  # why doesn't this work?
  return tf.strings.split(input_tensor, sep=',')

textVectorizer = TextVectorization(
                                  # output_sequence_length=max(n_genres_per_row), 
                                  #  split=split_by_comma_tf,
                                   pad_to_max_tokens=True,
                                   output_mode='binary')

In [165]:
textVectorizer.adapt(training_data)

In [166]:
vocab = textVectorizer.get_vocabulary()  # needs further cleaning (UNK, bl, '')
vocab[:10]

['[UNK]',
 'drama',
 'romance',
 'comedy',
 'mystery',
 'fantasy',
 'youth',
 'thriller',
 'melodrama',
 'family']

In [167]:
len(vocab)

38

In [168]:
model = tf.keras.models.Sequential()

In [169]:
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))

In [170]:
model.add(textVectorizer)

In [171]:
output = model.predict(training_data)
output[:5]

array([[0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]], dtype=float32)

In [172]:
output.shape

(111, 38)

# Calculate pairwise cosine similarity

In [173]:
cosine_loss = tf.keras.losses.CosineSimilarity(axis=1)

In [174]:
sample = output[:5, :]
cosine_loss(sample, sample).numpy()  # does not work with integer output (not binary)

-1.0

In [175]:
def compute_cosine_distances(a, b):
    # x shape is n_a * dim
    # y shape is n_b * dim
    # results shape is n_a * n_b

    normalize_a = tf.nn.l2_normalize(a,1)        
    normalize_b = tf.nn.l2_normalize(b,1)
    distance = 1 - tf.matmul(normalize_a, normalize_b, transpose_b=True)
    return distance

In [176]:
compute_cosine_distances(sample, sample)  # gives error with integers (not binary)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[5.9604645e-08, 1.0000000e+00, 5.5278641e-01, 1.0000000e+00,
        1.0000000e+00],
       [1.0000000e+00, 5.9604645e-08, 1.0000000e+00, 1.0000000e+00,
        5.9604645e-08],
       [5.5278641e-01, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
        1.0000000e+00],
       [1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        1.0000000e+00],
       [1.0000000e+00, 5.9604645e-08, 1.0000000e+00, 1.0000000e+00,
        5.9604645e-08]], dtype=float32)>

In [177]:
from sklearn.metrics.pairwise import cosine_similarity as sklearn_cosine_similarity
import numpy as np

In [178]:
cosine_simil = sklearn_cosine_similarity(output)

In [179]:
cosine_simil[:1, 1:]  # exlcuding first value as it's 1 (cosin similarity to itself)

array([[0.        , 0.4472136 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.31622776, 0.5163978 , 0.        ,
        0.        , 0.31622776, 0.        , 0.        , 0.        ,
        0.5163978 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 , 0.        , 0.        ,
        0.        , 0.2581989 , 0.        , 0.4472136 , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.2236068 ,
        0.        , 0.        , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.39999998, 0.        , 0.39999998,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.59999996, 0.2581989 , 0.2581989 ,
        0.77459663, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.5163978 , 0.        , 0.4472136 ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2581989 , 0.31622776, 0.        , 0.  

In [182]:
index_of_most_similar = np.argmax(cosine_simil[:1, 1:]) + 1

In [183]:
print(output[0])
print(output[index_of_most_similar])

[0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [184]:
print(df['main_name'].values[0])
print(df['main_name'].values[index_of_most_similar])

365: Repeat the Year
Missing: The Other Side


In [185]:
print(df['genre'].values[0])
print(df['genre'].values[index_of_most_similar])

 Thriller, fantasy, mystery, crime, time-travel
  Fantasy, mystery, thriller
